# ETL III - Transformación II - Clases y Funciones de limpieza

In [7]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import pyjsonviewer
import json

from datetime import datetime, timedelta

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [51]:
class energia:

    def __init__(self, inicio, fin):
        self.inicio = inicio
        self.fin = fin

    def tipos_energia(self):
        datos_energia = pd.DataFrame()

        for i in range(self.inicio, self.fin):

            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year"
            response = requests.get(url=url)

            # codigo_estado = response.status_code
            # razon_estado = response.reason

            # if codigo_estado == 200:
            #     print('Petición correcta, código de estado:',codigo_estado,'razón: ',razon_estado)
            # elif codigo_estado == 402:
            #     print('No se ha podido autorizar usario, código de estado:', codigo_estado,'razón: ',razon_estado)
            # elif codigo_estado == 404:
            #     print('Algo ha salido mal, código de estado:', codigo_estado,'razón: ',razon_estado)
            # else:
            #     print('Algo ha ocurrido, código de estado:', codigo_estado,'razón: ',razon_estado)

            response_jason = response.json()

            for e in range(len(response_jason["included"])):
                x = pd.DataFrame(response_jason["included"][e]["attributes"]["values"])
                x["tipo_energia"] = response_jason["included"][e]["attributes"]["title"]

                datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

        return datos_energia


    def comunidades_autonomas(self, codigo):
        self.codigo = codigo
        datos_energia = pd.DataFrame()

        for i in range(self.inicio, self.fin):
            for k,v in codigo.items():
            
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
                response = requests.get(url=url)

                # codigo_estado = response.status_code
                # razon_estado = response.reason

                # if codigo_estado == 200:
                #     print('Petición correcta, código de estado:',codigo_estado,'razón: ',razon_estado)
                # elif codigo_estado == 402:
                #     print('No se ha podido autorizar usario, código de estado:', codigo_estado,'razón: ',razon_estado)
                # elif codigo_estado == 404:
                #     print('Algo ha salido mal, código de estado:', codigo_estado,'razón: ',razon_estado)
                # else:
                #     print('Algo ha ocurrido, código de estado:', codigo_estado,'razón: ',razon_estado)

                respose_json = response.json()

                for e in range(len(respose_json["included"])):
                    x = pd.DataFrame(respose_json["included"][e]["attributes"]["values"])
                    x["tipo_energia"] = respose_json["included"][e]["attributes"]["title"]
                    x["comunidad"] = k
                    x["cod_comunidad"] = v

                    datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

        return datos_energia


    def redondear(self, df):
        self.df = df
        df['value'] = round(df['value'], 2)
        df['percentage'] = round(df['percentage'], 2)

        return df

    def extraer_fecha(self, df):
        self.df = df
        df['datetime'] = df['datetime'].apply(pd.to_datetime)
        df['datetime'] = df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))

        return df

    def guardado_datos(self, df, ruta, nombre):
        self.df =df
        self.ruta = ruta
        self.nombre = nombre

        df.to_pickle(ruta + nombre + ".pkl")
        df.to_csv(ruta + nombre + ".csv") 
    



In [52]:
codigo = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [53]:
energia(2011, 2023)

In [54]:
variable = energia(2011, 2023)

In [55]:
help(energia)

Help on class energia in module __main__:

class energia(builtins.object)
 |  energia(inicio, fin)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, inicio, fin)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  comunidades_autonomas(self, codigo)
 |  
 |  extraer_fecha(self, df)
 |  
 |  guardado_datos(self, df, ruta, nombre)
 |  
 |  redondear(self, df)
 |  
 |  tipos_energia(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [56]:
df_energias = variable.tipos_energia()

In [ ]:
df_comunidades = variable.comunidades_autonomas(codigo)

In [57]:
redondeo_energias = variable.redondear(df_energias)

In [58]:
redondeo_energias.head()

,value,percentage,datetime,tipo_energia
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,Renovable
1,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,No renovable
2,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,Renovable
3,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,No renovable
4,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,Renovable


In [ ]:
redondeo_comunidades = variable.redondear(df_comunidades)

In [ ]:
df_fechas_tipos = variable.extraer_fecha(df_energias)
df_fechas_tipos

In [ ]:
df_fechas_comunidades = variable.extraer_fecha(df_comunidades)

In [ ]:
fichero_tipos = variable.guardado_datos(df_fechas_tipos, "datos/", "tipo_energia")

In [ ]:
fichero_comunidades = variable.guardado_datos(df_fechas_comunidades, "datos/", "datos_comunidades")